In [3]:
from google.colab import files
uploaded = files.upload()




Saving animal_classification.zip to animal_classification (1).zip


In [4]:
import zipfile
import os

# Path to the uploaded file
zip_file_path = 'animal_classification.zip'  # Ensure this matches your uploaded file name
extract_dir = '/content/animal_classification/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List extracted files
print("Extracted files:", os.listdir(extract_dir))


Extracted files: ['Animal Classification']


In [5]:
dataset_path = '/content/animal_classification/'
print("Files and folders:", os.listdir(dataset_path))


Files and folders: ['Animal Classification']


In [6]:
dataset_path = '/content/animal_classification/Animal Classification/dataset/'  # Adjust if required
print("Animal classes:", os.listdir(dataset_path))


Animal classes: ['Deer', 'Kangaroo', 'Zebra', 'Lion', 'Horse', 'Bird', 'Elephant', 'Giraffe', 'Panda', 'Tiger', 'Cow', 'Cat', 'Dog', 'Bear', 'Dolphin']


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define parameters
image_size = (224, 224)  # Resize all images to 224x224
batch_size = 32          # Number of images processed at once

# Create a data generator
data_gen = ImageDataGenerator(
    rescale=1.0 / 255,    # Normalize pixel values to [0, 1]
    validation_split=0.2  # 20% of data used for validation
)

# Load training and validation data
train_data = data_gen.flow_from_directory(
    '/content/animal_classification/Animal Classification/dataset/',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42  # Ensures reproducibility
)

val_data = data_gen.flow_from_directory(
    '/content/animal_classification/Animal Classification/dataset/',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42
)

# Print class indices
print("Class indices:", train_data.class_indices)


Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.
Class indices: {'Bear': 0, 'Bird': 1, 'Cat': 2, 'Cow': 3, 'Deer': 4, 'Dog': 5, 'Dolphin': 6, 'Elephant': 7, 'Giraffe': 8, 'Horse': 9, 'Kangaroo': 10, 'Lion': 11, 'Panda': 12, 'Tiger': 13, 'Zebra': 14}


In [8]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_data.class_indices), activation='softmax')  # Output layer for 15 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,170,895 (42.61 MB)

 Trainable params: 11,170,895 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  # Adjust epochs based on your preference
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/49 ━━━━━━━━━━━━━━━━━━━━ 208s 4s/step - accuracy: 0.0832 - loss: 3.3890 - val_accuracy: 0.1514 - val_loss: 2.5844
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 197s 4s/step - accuracy: 0.1897 - loss: 2.5002 - val_accuracy: 0.2898 - val_loss: 2.2952
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.4214 - loss: 1.8926 - val_accuracy: 0.3185 - val_loss: 2.5452
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 204s 4s/step - accuracy: 0.6552 - loss: 1.2402 - val_accuracy: 0.3211 - val_loss: 2.8943
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 197s 4s/step - accuracy: 0.8675 - loss: 0.5970 - val_accuracy: 0.3316 - val_loss: 3.9116
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9277 - loss: 0.2971 - val_accuracy: 0.3055 - val_loss: 5.0636
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 206s 4s/step - accuracy: 0.9830 - loss: 0.1100 - val_accuracy: 0.3446 - val_loss: 5.5564
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 199s 4s/step - accuracy: 0.9867 - loss: 0.0999 - val_accuracy: 0.3786 - val_loss: 7.

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to your sample image
test_image_path = '/content/animal_classification/Animal Classification/dataset/Lion/Lion_1.jpg'

# Preprocess the image
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = np.expand_dims(image.img_to_array(img) / 255.0, axis=0)

# Make a prediction
prediction = model.predict(img_array)
predicted_class = list(train_data.class_indices.keys())[np.argmax(prediction)]
print(f"Predicted Class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Predicted Class: Lion


In [12]:
model.save('animal_classification_model.h5')
print("Model saved as animal_classification_model.h5")


Model saved as animal_classification_model.h5


In [1]:
test_image_path = '/content/animal_classification/Animal Classification/dataset/Tiger/Tiger_1.jpg'

# Preprocess the image
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = np.expand_dims(image.img_to_array(img) / 255.0, axis=0)

# Make a prediction
prediction = model.predict(img_array)
predicted_class = list(train_data.class_indices.keys())[np.argmax(prediction)]
print(f"Predicted Class: {predicted_class}")


NameError: name 'image' is not defined